# BDM group project  - Targeting Model - Group D 
 

Group members:

 * Matej Federic: M20210118
 * Victor Almeida: M20210270
 * Andrea Verbaro: M20210053

# Summary

## Business request
After analysing the Home AB test, you’ve found out that the main difference
between the two homes is the List 3 component. Based on the A/B testing we want to boost the performance knowing what home version we should
show to each customer, individually. Hence we would need to develop a Recommendation / targeting model to pin point what are the customers with higher probability
to convert given the home list 3 to be seen.

## Contents <br><br>

 * 1 - Importing Dataframes
 * 2 - Data preparation  - creation of KPIS
 * 3 - Pipeline for data cleaning and preparation
 * 4 - Modelling
 * 5 - Evaluation of the model and optimization
 * 6 - Evaluation on the April dataset
 * 7 - Performance evaluation
 * 8 - Final comment

# 1 - Importing Dataframes

In [0]:
# imports
import pyspark.pandas as ps
from pyspark.sql.window import Window
from pyspark.sql import functions as f
from pyspark.sql.types import StringType, ArrayType, LongType, DateType, BooleanType, StructType, StructField

In [0]:
spark.sparkContext.setLogLevel("WARN")

In [0]:
%sh 
wget https://www.dropbox.com/s/0prlh78825xy4tk/BDM_DATA.zip --quiet
unzip -d ./bdm_data/ BDM_DATA.zip


Archive:  BDM_DATA.zip
   creating: ./bdm_data/cust_df/
  inflating: ./bdm_data/cust_df/_SUCCESS  
  inflating: ./bdm_data/cust_df/_committed_7878371389005906564  
  inflating: ./bdm_data/cust_df/_committed_5076822134895256271  
  inflating: ./bdm_data/cust_df/_committed_3520508812338357534  
  inflating: ./bdm_data/cust_df/_started_5076822134895256271  
  inflating: ./bdm_data/cust_df/part-00000-tid-5076822134895256271-fa5ebda2-8174-4bce-b5ef-fec6a0376257-23668-1-c000.csv  
   creating: ./bdm_data/orders_df/
  inflating: ./bdm_data/orders_df/_committed_1749678841354862380  
  inflating: ./bdm_data/orders_df/_committed_4173638812266093034  
  inflating: ./bdm_data/orders_df/_committed_4196442019492113282  
  inflating: ./bdm_data/orders_df/_committed_1102646042990821830  
  inflating: ./bdm_data/orders_df/_started_568900438245366187  
  inflating: ./bdm_data/orders_df/_committed_568900438245366187  
  inflating: ./bdm_data/orders_df/part-00004-tid-568900438245366187-3cd9d01f-3962-4d69-

In [0]:
# The folder "/FileStore" on DBFS is the default folder for imported data.  
# dbutils.fs.mv("file:/databricks/driver/bdm_data/", "dbfs:/FileStore/bdm_data/", True)

Out[9]: True

## 1.1 - Import of data

In [0]:
sessions_df_schema = StructType([
    StructField("customer_id",StringType(),True),
    StructField(
        "session_events",
        ArrayType(
            StructType([
                StructField("datetime", StringType(),True),
                StructField("event", StringType(),True)
            ]), True)
    ),
    StructField("session_id", StringType() ,True),
    StructField("session_rank", LongType(), True)
])

sessions_df = (
    spark.read.format("json")
    .schema(sessions_df_schema)
    .load("dbfs:/FileStore/bdm_data/sessions_df")
    .select('customer_id', 'session_events', 'session_id')
    .toDF('customer_id', 'session_events', 'session_id')
    .withColumn("session_events", f.explode('session_events'))
    .select(f.col("customer_id"), f.col("session_events.*"), f.col("session_id"))
    .withColumn('session_timestamp', f.col('datetime').cast('timestamp'))
    .withColumn('session_date', f.to_date(f.col('datetime')))
    .withColumn('session_date_month', f.date_trunc('month', f.col('datetime')).cast('date'))
    .withColumn('session_hour', f.hour(f.col('datetime')))
)

w_lag   = Window.partitionBy(f.col("customer_id")).orderBy(f.col("order_timestamp"))
w_month = Window.partitionBy(f.col("customer_id"), f.col('month')).orderBy(f.col("order_timestamp"))
 
orders_df =(
    spark.read
    .format("parquet").option("inferSchema", "true")
    # Reading the orders parquet file from DBFS
    .load("dbfs:/FileStore/bdm_data/orders_df/")
    # Renaming columns
    .toDF('order_id', 'session_id', 'order_timestamp', 'customer_id', 'total_value', 'discount_value', 'order_category')
    # Creating a column with paid value info
    .withColumn('paid_value', f.col('total_value') - f.col('discount_value'))
    # Forcing the time of the purchase to timestamp data type
    .withColumn('order_timestamp', f.col('order_timestamp').astype('timestamp'))
    # Extracting the hour of the timestamp
    .withColumn('hour', f.hour(f.col('order_timestamp')))
    # Extracting the month of the timestamp
    .withColumn('month', f.date_trunc('month', f.col('order_timestamp')).astype('date') )
    # Creating the shift
    .withColumn('shift', f.when(f.col('hour') <= 10, 'breakfast').when(f.col('hour')<=17, 'lunch' ).otherwise('dinner'))
    # Creating the discount range category
    .withColumn('discount_percentage', f.round(f.col('discount_value') / f.col('total_value'), 2) )
    .withColumn('discount_range', 
                f.when(f.col('discount_percentage') <= 0.10, '0-10%')
                .when(f.col('discount_percentage') <= 0.20, '10-20%')
                .otherwise('30%+'))
    .withColumn("order_date_lag", f.lag("order_timestamp", offset=1, default=None).over(w_lag))
    .withColumn("days_since_last_order", f.datediff(f.col('order_timestamp'), f.col('order_date_lag')))
    .withColumn("last_order", f.max("order_timestamp").over(w_lag))
    .withColumn("recency", f.datediff(f.current_date(), f.col('last_order')))
)

cust_df = (
    spark.read
    .format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .load("dbfs:/FileStore/bdm_data/cust_df/")
    .select('customer_id', f.col('is_referee').astype('float'), 'device_type', 'install_origin')
)


In [0]:
# prepare the kpis based on orders 
# 0000be59-44b8-44a7-85dc-b5c417ba2858

orders_df\
    .where(f.col('customer_id') == '0000be59-44b8-44a7-85dc-b5c417ba2858')\
    .limit(15).display()


order_id,session_id,order_timestamp,customer_id,total_value,discount_value,order_category,paid_value,hour,month,shift,discount_percentage,discount_range,order_date_lag,days_since_last_order,last_order,recency,feb_expenses
88c02423-ccb6-480a-8f24-743493de6a3b,111c756f-cc33-42f0-8e8f-729a4a591f9b,2021-09-09T21:19:31.180+0000,0000be59-44b8-44a7-85dc-b5c417ba2858,73.3,0.0,Alc Beverages,73.3,21,2021-09-01,dinner,0.0,0-10%,null,null,2021-09-09T21:19:31.180+0000,282,0.0
1d485b18-d6f5-4c6e-ac48-886034ef126e,581dfc0f-0abd-47cd-8c59-4a0defa0588c,2021-09-23T20:16:20.723+0000,0000be59-44b8-44a7-85dc-b5c417ba2858,73.8,0.0,Alc Beverages,73.8,20,2021-09-01,dinner,0.0,0-10%,2021-09-09T21:19:31.180+0000,14,2021-09-23T20:16:20.723+0000,268,0.0
f22c3eac-847c-4a5f-964f-0d293f07a5e3,10ddd31f-8f41-4e4b-b99f-2b6133f16f4e,2021-09-27T22:27:27.704+0000,0000be59-44b8-44a7-85dc-b5c417ba2858,72.9,0.0,Alc Beverages,72.9,22,2021-09-01,dinner,0.0,0-10%,2021-09-23T20:16:20.723+0000,4,2021-09-27T22:27:27.704+0000,264,0.0
f76d1e63-66fc-4a89-a619-61bdc9cdb3ff,74ef009f-a3fe-43ae-8cff-bf43bc294a33,2021-10-05T20:47:36.575+0000,0000be59-44b8-44a7-85dc-b5c417ba2858,74.2,0.0,Burger,74.2,20,2021-10-01,dinner,0.0,0-10%,2021-09-27T22:27:27.704+0000,8,2021-10-05T20:47:36.575+0000,256,0.0
6f043389-f5e8-4b9d-bfdb-cb99623a7833,5756e15a-7798-40a9-873f-ef1d02d9399b,2021-11-10T22:39:45.589+0000,0000be59-44b8-44a7-85dc-b5c417ba2858,54.0,0.0,Japanese,54.0,22,2021-11-01,dinner,0.0,0-10%,2021-10-05T20:47:36.575+0000,36,2021-11-10T22:39:45.589+0000,220,0.0
a19cf547-6c8c-4d37-b0d0-ff1dc0ea554c,e7ac6487-6f1d-473e-9472-af373565c532,2022-01-23T15:55:08.235+0000,0000be59-44b8-44a7-85dc-b5c417ba2858,66.9,0.0,Alc Beverages,66.9,15,2022-01-01,lunch,0.0,0-10%,2021-11-10T22:39:45.589+0000,74,2022-01-23T15:55:08.235+0000,146,0.0
fa43e8dc-f34e-4d5e-9cad-f250e6e5c24a,def0d910-eabf-4b32-bf5c-07fd7ae2b6e5,2022-02-04T11:26:45.888+0000,0000be59-44b8-44a7-85dc-b5c417ba2858,51.5,0.0,Alc Beverages,51.5,11,2022-02-01,lunch,0.0,0-10%,2022-01-23T15:55:08.235+0000,12,2022-02-04T11:26:45.888+0000,134,51.5
e991eac0-406e-4454-acf7-701e5766c336,061389bc-4a6e-4e02-827d-2660f116c8c7,2022-02-13T17:15:35.425+0000,0000be59-44b8-44a7-85dc-b5c417ba2858,79.4,0.0,Alc Beverages,79.4,17,2022-02-01,lunch,0.0,0-10%,2022-02-04T11:26:45.888+0000,9,2022-02-13T17:15:35.425+0000,125,130.9000015258789
ab89fc43-d77e-4431-aae9-531cb4e8ca5f,9627927f-064a-4773-b3f6-1bef3f3232eb,2022-02-17T20:53:22.434+0000,0000be59-44b8-44a7-85dc-b5c417ba2858,69.1,0.0,Alc Beverages,69.1,20,2022-02-01,dinner,0.0,0-10%,2022-02-13T17:15:35.425+0000,4,2022-02-17T20:53:22.434+0000,121,200.0
3a3461b5-62ca-4d72-ada5-2b88dde2e89c,c366591f-4d0c-480d-b023-06983300f132,2022-02-20T23:26:16.236+0000,0000be59-44b8-44a7-85dc-b5c417ba2858,76.0,0.0,Burger,76.0,23,2022-02-01,dinner,0.0,0-10%,2022-02-17T20:53:22.434+0000,3,2022-02-20T23:26:16.236+0000,118,276.0


# 2- Data Preparation

## Creation of the variables <br><br>
**From Session_df**

 * Session_date_month = month of session --> needed for the final dataset
 * Session_hour = hour of session
 * Open_tmsp = timestamp of beginning of session 
 * Close_tmsp = timestamp of end of session 

 * Converted = conversions given by "CallbackPurcahse Event in sessions"
 * Cvr = Converstion rate for sessions
 * Drop_rate = 1 - CVR
 * conversion = absolute value of conversions

**From order_df** 
 * paid_value  = (total_value) minus (Discount_value)
 * Hour = hour of the order
 * Month = month of the order
 * Shift – (variables breakfast, lunch and dinner)
 * Discount percentage = (discount_value) / (total_value)
 * Days_since_last_order  = time in days from last order 
 * Avg_elapsed_time = time elapsed between each orders
 * Avg_hour = average hour of session
 * Frequency = frequency of purchase
 * Tot_discount_percentage = tot discount percentage given all expenses
 * Tot_gross_value = tot gross values paid
 * Avg_expenses = average gross expenses
 * Avg_discount = average discount in total values
 * Tot_net_paid = total net paid purchase 
 * Avg_paid_value = averge net paid values
 * Avg_discount_percentage = avg disocunt percentage
 * Avg_shift = most common shift netween lunch, dinner and 
 * Last3_months_exp = tot expense for last 3 months
 * Last3_month_avg_exp = average expense for last 3 months
 * Purchase by category = Pizza, alcoholics, Vegetarian, Japanese, Burger expresseed in percentages

In [0]:
# create the expenses by month 
w_month = Window.partitionBy(f.col("customer_id"), f.col('month')).orderBy(f.col("month"))
w_cust = Window.partitionBy(f.col("customer_id")).orderBy(f.col("customer_id"))
w_cust_cumul = (Window.partitionBy('customer_id').orderBy('month').rangeBetween(Window.unboundedPreceding, 0))

orders_input_1 = (
    orders_df\
        .select(f.col('customer_id'), f.col('total_value'), f.col('month'), f.col('hour'),
                f.col('discount_value'), f.col('days_since_last_order'), f.col('recency') )\
        .where(f.col('month') >= '2022-02-01')\
        .groupBy(f.col('customer_id'), f.col('month'))\
        .agg(
            f.avg('days_since_last_order').alias('avg_elapsed_time'),
            f.round(f.avg('hour'),0).alias('avg_hour'),
            f.count(f.col('total_value')).alias('frequency'),
            f.sum('discount_value').alias('tot_discount_percentage'),
            f.sum('total_value').alias('tot_gross_value'),
            f.avg('total_value').alias('avg_expenses'),
            f.avg('discount_value').alias('avg_discount')
        )
        .withColumn('tot_net_paid', f.col('tot_gross_value') - f.col('tot_discount_percentage'))\
        .withColumn('avg_paid_value', f.col('avg_expenses') - f.col('avg_discount'))\
        .withColumn('avg_discount_percentage', f.col('tot_discount_percentage') / f.col('tot_gross_value'))\
        .withColumn('avg_shift', f.when(f.col('avg_hour') <= 10, 'breakfast').when(f.col('avg_hour')<=17, 'lunch' ).otherwise('dinner'))\
        .withColumn('last3_months_exp', f.sum('tot_net_paid').over(w_cust_cumul))\
        .withColumn('last3_months_avg_exp', f.avg('avg_paid_value').over(w_cust_cumul))
)  

In [0]:
w_session = Window.partitionBy(f.col("customer_id"), f.col('session_id')).orderBy(f.col("session_id"))

session_duration = (
        sessions_df
            .filter(f.col('session_date_month')>='2022-02-01')
            .select(f.col('session_id'), f.col('customer_id'), f.col('session_timestamp'), f.col('session_date_month'))
            .withColumn('open_tmsp',f.min(f.col('session_timestamp')).over(w_session))
            .withColumn('close_tmsp',f.max(f.col('session_timestamp')).over(w_session))
            .withColumn('duration_sec',f.col("close_tmsp").cast("long") - f.col('open_tmsp').cast("long"))
            .groupBy('customer_id','session_date_month')
            .agg(f.avg('duration_sec').alias('avg_navig_seconds'))
            .select(f.col('customer_id'), f.col('session_date_month').alias('month'), f.col('avg_navig_seconds').alias('duration_sec'))
    
       )

In [0]:
session_duration.limit(5).display()

customer_id,session_date_month,avg_navig_seconds
0039e757-e52d-4e39-90e4-b01a11b623b9,2022-04-01,107.13636363636364
009a901a-4079-4bde-9f58-833de2edce51,2022-03-01,116.41891891891892
009bb52d-62f9-4742-ac4a-21fdf300328b,2022-04-01,114.11111111111111
009f5c31-722e-474e-a7e7-6c341ad0ec4f,2022-04-01,118.28682170542636
00acd007-767b-427a-a09a-202a27e450d6,2022-03-01,117.6


In [0]:
session_duration_example = (
        sessions_df
            .filter(f.col('session_date_month')>='2022-02-01')
            .select(f.col('session_id'), f.col('customer_id'), f.col('session_timestamp'), f.col('session_date_month'))
            .groupBy('customer_id','session_date_month', 'session_id')
            .agg(
                f.min(f.col('session_timestamp')).alias('open_tmsp'),
                f.max(f.col('session_timestamp')).alias('close_tmsp')
            )
            .withColumn('duration_sec',f.col("close_tmsp").cast("long") - f.col('open_tmsp').cast("long"))
            .groupBy('customer_id','session_date_month')
            .agg(
                    f.avg('duration_sec').alias('avg_navig_seconds')
            )
           
       )

In [0]:
session_duration_example.limit(10).display()

customer_id,session_date_month,avg_navig_seconds
6851891a-c7a0-474e-9155-9f01e7d704a7,2022-04-01,110.625
fc4316c6-34b3-44a7-9ed7-b93f5a2a1f44,2022-02-01,116.53846153846153
1bd5d942-50b0-448d-abf8-c19d1ed575fc,2022-03-01,110.13793103448276
47377728-7024-47f2-8af5-6ff9111b4e74,2022-03-01,115.05555555555556
2352ecf9-6193-49d1-9712-54586f40df14,2022-02-01,97.14285714285714
22c8d25e-056e-4311-a87f-5872dd33e970,2022-04-01,110.0
f806ddf3-5e26-4c72-b10e-ef4da0616ae5,2022-04-01,97.55555555555556
c1bc7dbb-4163-4351-b5ec-1f4830bd0408,2022-02-01,113.60714285714286
8f1d4c31-8dbc-44cb-909d-acbe3bdcce6d,2022-04-01,112.625
1d0855da-046b-47eb-8479-91f3d208695e,2022-04-01,117.46153846153847


In [0]:
sessions_input = (
    sessions_df
        .where(f.col('session_date_month') >='2022-02-01')
        .select(f.col('customer_id'), f.col('session_id'), f.col('session_date_month'))
        .groupBy('customer_id','session_date_month')
        .agg(
            f.countDistinct('session_id').alias('freq_sessions'))
        .select(f.col('customer_id'), f.col('session_date_month').alias('month'), f.col('freq_sessions'))
)


In [0]:
orders_input_2 = (
    orders_df\
        .where(f.col('month') >= '2022-02-01')\
        .select(f.col('order_id'), f.col('customer_id'), f.col('order_category'), f.col('month'), f.col('total_value'),
               f.col('discount_value'))\
        .withColumn('paid_value', f.col('total_value') - f.col('discount_value'))\
        .groupby(f.col('customer_id'), f.col('month'))\
        .agg(
            f.sum(f.when( (f.col('order_category')=='Pizza'), f.lit(1)).otherwise(f.lit(0)) ).cast('float') .alias('Pizza'),
            f.sum(f.when( (f.col('order_category')=='Burger'), f.lit(1)).otherwise(f.lit(0)) ).cast('float') .alias('Burger'),
            f.sum(f.when( (f.col('order_category')=='Alc Beverages'), f.lit(1)).otherwise(f.lit(0)) ).cast('float') .alias('Alcohol'),
            f.sum(f.when( (f.col('order_category')=='Japanese'), f.lit(1)).otherwise(f.lit(0)) ).cast('float') .alias('Japanese'),
            f.sum(f.when( (f.col('order_category')=='Vegetarian'), f.lit(1)).otherwise(f.lit(0)) ).cast('float') .alias('Veggy'),    
        )
        .withColumn('tot_values' , f.col('Pizza').cast('float') + f.col('Burger').cast('float') 
                    + f.col('Alcohol').cast('float')  + f.col('Japanese').cast('float')  + f.col('Veggy').cast('float') )\
        .withColumn('%Pizza', f.round(f.col('Pizza').cast('float')  / f.col('tot_values'),2))\
        .withColumn('%Burger', f.round(f.col('Burger').cast('float')  / f.col('tot_values'),2))\
        .withColumn('%Japanese',f.round( f.col('Japanese').cast('float')  / f.col('tot_values'),2))\
        .withColumn('%Alcohol', f.round(f.col('Alcohol').cast('float')  / f.col('tot_values'),2))\
        .withColumn('%Veggy', f.round(f.col('Veggy').cast('float')  / f.col('tot_values'),2))\
)

In [0]:
orders_input_2.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- month: date (nullable = true)
 |-- Pizza: float (nullable = true)
 |-- Burger: float (nullable = true)
 |-- Alcohol: float (nullable = true)
 |-- Japanese: float (nullable = true)
 |-- Veggy: float (nullable = true)
 |-- tot_values: float (nullable = true)
 |-- %Pizza: double (nullable = true)
 |-- %Burger: double (nullable = true)
 |-- %Japanese: double (nullable = true)
 |-- %Alcohol: double (nullable = true)
 |-- %Veggy: double (nullable = true)



In [0]:
# this table not used
orders_input_3 = (
    orders_df\
        .where(f.col('month') >= '2022-02-01')\
        .select(f.col('order_id'), f.col('customer_id'), f.col('order_category'), f.col('month'), f.col('total_value'),
               f.col('discount_value'))\
        .withColumn('paid_value', f.col('total_value') - f.col('discount_value'))\
        .groupby(f.col('customer_id'), f.col('month'))\
        .agg( 
            f.coalesce(
               f.sum(f.when( (f.col('order_category')=='Pizza'), 'paid_value').otherwise(0) ),
                f.lit(0)
            ).cast('float').alias('Pizza€'),
            f.coalesce(
                f.sum(f.when( (f.col('order_category')=='Burger'), 'paid_value').otherwise(0) ),
               f.lit(0)
            ).cast('float').alias('Burger€'),
            f.coalesce(
                f.sum(f.when( (f.col('order_category')=='Alc Beverages'), 'paid_value').otherwise(0) ),
                f.lit(0)
            ).cast('float').alias('Alcohol€'),
            f.coalesce(
                f.sum(f.when( (f.col('order_category')=='Japanese'), 'paid_value').otherwise(0) ),
                f.lit(0)
            ).cast('float').alias('Japanese€'),
            f.coalesce(
                f.sum(f.when( (f.col('order_category')=='Vegetarian'), 'paid_value').otherwise(0 )),
                f.lit(0)
            ).cast('float').alias('Veggy€')
         )
        .withColumn('tot_spent' , f.col('Pizza€') + f.col('Burger€') + f.col('Alcohol€') + f.col('Japanese€') + f.col('Veggy€'))\
        .withColumn('%Pizza€', f.col('Pizza€') / f.col('tot_spent'))\
        .withColumn('%Burger€', f.col('Burger€') / f.col('tot_spent'))\
        .withColumn('%Japanese€', f.col('Japanese€') / f.col('tot_spent'))\
        .withColumn('%Alcohol€', f.col('Alcohol€') / f.col('tot_spent'))\
        .withColumn('%Veggy€', f.col('Veggy€') / f.col('tot_spent'))\
)

In [0]:
orders_input_2.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- month: date (nullable = true)
 |-- Pizza: float (nullable = true)
 |-- Burger: float (nullable = true)
 |-- Alcohol: float (nullable = true)
 |-- Japanese: float (nullable = true)
 |-- Veggy: float (nullable = true)
 |-- tot_values: float (nullable = true)
 |-- %Pizza: double (nullable = true)
 |-- %Burger: double (nullable = true)
 |-- %Japanese: double (nullable = true)
 |-- %Alcohol: double (nullable = true)
 |-- %Veggy: double (nullable = true)



In [0]:
conversions_input = (
    sessions_df
        .where(f.col('session_date_month') >='2022-02-01')
        .select(f.col('customer_id'), f.col('session_date_month'), f.col('session_id'), f.col('event'))
        .withColumn('converted', f.when(f.col('event') == 'CallbackPurchase', f.lit(1)).otherwise(f.lit(0)))
        .groupBy('customer_id','session_date_month')
        .agg(
            f.countDistinct('session_id').alias('freq_sessions'),
            f.sum('converted').alias('conversions'))
        .withColumn('cvr', f.col('conversions') / f.col('freq_sessions'))
        .withColumn('drop_rate', 1 - f.col('cvr'))
        .select(f.col('customer_id'), f.col('session_date_month').alias('month')
                , f.col('conversions'), f.col('cvr'), f.col('drop_rate'))
    )



In [0]:
sessions_df.limit(10).display()

customer_id,datetime,event,session_id,session_timestamp,session_date,session_date_month,session_hour
828d7bdf-96eb-4e61-af45-69dcc8ec73be,2022-04-17 13:37:15.957171,OpenApp,c6be7a50-cb0c-4c0c-81ca-3ed39c093718,2022-04-17T13:37:15.957+0000,2022-04-17,2022-04-01,13
828d7bdf-96eb-4e61-af45-69dcc8ec73be,2022-04-17 13:37:27.126802,ViewHome,c6be7a50-cb0c-4c0c-81ca-3ed39c093718,2022-04-17T13:37:27.126+0000,2022-04-17,2022-04-01,13
828d7bdf-96eb-4e61-af45-69dcc8ec73be,2022-04-17 13:38:47.358932,ViewSearch,c6be7a50-cb0c-4c0c-81ca-3ed39c093718,2022-04-17T13:38:47.358+0000,2022-04-17,2022-04-01,13
828d7bdf-96eb-4e61-af45-69dcc8ec73be,2022-04-17 13:38:40.752893,CloseApp,c6be7a50-cb0c-4c0c-81ca-3ed39c093718,2022-04-17T13:38:40.752+0000,2022-04-17,2022-04-01,13
828d7bdf-96eb-4e61-af45-69dcc8ec73be,2022-03-31 18:32:15.958422,OpenApp,5c3a3b8b-4f4b-4cbd-a7e5-5a23e54fb31b,2022-03-31T18:32:15.958+0000,2022-03-31,2022-03-01,18
828d7bdf-96eb-4e61-af45-69dcc8ec73be,2022-03-31 18:32:24.135571,ViewHomeVariant,5c3a3b8b-4f4b-4cbd-a7e5-5a23e54fb31b,2022-03-31T18:32:24.135+0000,2022-03-31,2022-03-01,18
828d7bdf-96eb-4e61-af45-69dcc8ec73be,2022-03-31 18:33:44.611742,ViewList3,5c3a3b8b-4f4b-4cbd-a7e5-5a23e54fb31b,2022-03-31T18:33:44.611+0000,2022-03-31,2022-03-01,18
828d7bdf-96eb-4e61-af45-69dcc8ec73be,2022-03-31 18:33:50.718991,CloseApp,5c3a3b8b-4f4b-4cbd-a7e5-5a23e54fb31b,2022-03-31T18:33:50.718+0000,2022-03-31,2022-03-01,18
828d7bdf-96eb-4e61-af45-69dcc8ec73be,2022-03-21 14:42:15.959261,OpenApp,a7547d25-f265-4457-b915-234c8732f642,2022-03-21T14:42:15.959+0000,2022-03-21,2022-03-01,14
828d7bdf-96eb-4e61-af45-69dcc8ec73be,2022-03-21 14:42:19.472056,ViewHomeVariant,a7547d25-f265-4457-b915-234c8732f642,2022-03-21T14:42:19.472+0000,2022-03-21,2022-03-01,14


In [0]:
sessions_df.select('event').distinct().display()

event
ViewSearch
ViewList3
ClickAddRecommendedItem
ViewCartCheckout
CallbackPurchase
ViewHomeVariant
CallbackDonation
CloseApp
OpenApp
ViewHome


In [0]:
# add the filter only for customers who saw item list 3
customers_viewlist3 = (
    sessions_df
        .where(f.col('session_date_month') >= '2022-02-01')\
        .withColumn('view_list3', f.when(f.col('event') == 'ViewList3', f.lit(1)).otherwise(f.lit(0)))\
        .withColumn('converted', f.when(f.col('event') == 'CallbackPurchase', f.lit(1)).otherwise(f.lit(0)))\
        .groupBy( f.col('customer_id'), f.col('session_date_month'))\
        .agg(
            f.sum(f.col('view_list3')).alias('view_list3'),
            f.sum(f.col('converted')).alias('bought')
        )
        .where(f.col('view_list3')>0)\
        .drop('view_list3')\
        .withColumn('response', f.when(f.col('bought') > 0, f.lit(1)).otherwise(0))\
        .select(f.col('customer_id'), f.col('session_date_month').alias('month'), f.col('bought'), f.col('response'))
    )


In [0]:
customers_viewlist3.count()

Out[83]: 205384

In [0]:
customers_viewlist3\
    .where(f.col('bought') > 0)\
    .limit(10).display()

customer_id,session_date_month,bought,reponse
5155bce8-0c8d-419f-85d2-8ed6f01980de,2022-03-01,10,1
736065e5-207d-4875-9f06-1e58fab8dd0e,2022-02-01,1,1
6851891a-c7a0-474e-9155-9f01e7d704a7,2022-04-01,11,1
01cef08b-acc3-4b3e-a7a1-d3f38b35e685,2022-04-01,6,1
cf15a938-856a-4835-b365-276520297daa,2022-03-01,1,1
fe07e930-2cb6-42fa-8907-d5f202365077,2022-03-01,13,1
48f20e66-3b20-4c98-9e6b-20208abe67de,2022-04-01,5,1
d2bf6663-bd6a-44fc-8cfb-f84e9e2bfd22,2022-03-01,12,1
ec310451-da4f-48e9-bc92-e5979e285ede,2022-03-01,3,1
0c62e253-6db7-4ab7-8951-0cc77ae91187,2022-03-01,4,1


In [0]:
# check the amount of rows for each dataset to understand the results of the join 
for i in [conversions_input, orders_input_1, orders_input_2, sessions_input, orders_input_3, customers_viewlist3]:
    print(f'Rows count str(i) is  {i.count()}')

Rows count str(i) is  281751
Rows count str(i) is  199451
Rows count str(i) is  199451
Rows count str(i) is  281751
Rows count str(i) is  199451
Rows count str(i) is  205384


In [0]:
type(conversions_input)

Out[95]: pyspark.sql.dataframe.DataFrame

In [0]:
# creation of input data
input_data = (
    conversions_input
    .join(orders_input_1, ['customer_id', 'month'], 'inner')
    .join(orders_input_2, ['customer_id', 'month'], 'inner')
    .join(sessions_input, ['customer_id', 'month'], 'inner')
    .join(customers_viewlist3, ['customer_id', 'month'], 'inner')
    .join(cust_df, ['customer_id'], 'inner')
    .join(session_duration, ['customer_id', 'month'], 'inner')
)


In [0]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS project_data")
#input_data.write.mode('overwrite').option("header", "true").saveAsTable("project_data.input_data")
input_data.write.mode('overwrite').option("overwriteSchema", "true").option("header", "true").saveAsTable("project_data.input_data_v2")
input_df = spark.table("project_data.input_data_v2")


### 2.1 - Check on missing values

In [0]:
def count_missings(spark_df,sort=True):
    """
    Counts number of nulls and nans in each column
    """
    df = spark_df.select([f.count(f.when(f.isnan(c) | f.isnull(c), c)).alias(c) for (c,c_type) in spark_df.dtypes if c_type not in ('timestamp', 'string', 'date')]).toPandas()

    if len(df) == 0:
        print("There are no any missing values!")
        return None

    if sort:
        return df.rename(index={0: 'count'}).T.sort_values("count",ascending=False)

    return df


count_missings(input_df)


,count
avg_elapsed_time,3986
conversions,0
Burger,0
is_referee,0
response,0
bought,0
freq_sessions,0
%Veggy,0
%Alcohol,0
%Japanese,0


In [0]:
input_df.limit(100).display()

customer_id,month,conversions,cvr,drop_rate,avg_elapsed_time,avg_hour,frequency,tot_discount_percentage,tot_gross_value,avg_expenses,avg_discount,tot_net_paid,avg_paid_value,avg_discount_percentage,avg_shift,last3_months_exp,last3_months_avg_exp,Pizza,Burger,Alcohol,Japanese,Veggy,tot_values,%Pizza,%Burger,%Japanese,%Alcohol,%Veggy,freq_sessions,bought,response,is_referee,device_type,install_origin,duration_sec
0000be59-44b8-44a7-85dc-b5c417ba2858,2022-03-01,7,0.3888888888888889,0.6111111111111112,3.5454545454545454,18.0,11,0.0,656.4000015258789,59.67272741144354,0.0,656.4000015258789,59.67272741144354,0.0,dinner,932.4000015258789,64.33636370572177,0.0,3.0,6.0,0.0,2.0,11.0,0.0,0.27,0.0,0.55,0.18,18,7,1,0.0,Low-End,Email,113.75757575757575
00089da8-33fb-4971-9ffc-92cf9bd159e7,2022-04-01,6,1.0,0.0,6.0,18.0,3,0.0,110.70000076293945,36.90000025431315,0.0,110.70000076293945,36.90000025431315,0.0,dinner,617.5,48.166666666666664,0.0,3.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,6,6,1,0.0,High-End,Meta,117.04255319148936
00110e08-8611-4fb4-9773-c27fc617bff6,2022-02-01,12,0.5,0.5,2.625,19.0,8,0.0,703.6000061035156,87.95000076293945,0.0,703.6000061035156,87.95000076293945,0.0,dinner,703.6000061035156,87.95000076293945,0.0,3.0,0.0,5.0,0.0,8.0,0.0,0.38,0.63,0.0,0.0,24,12,1,0.0,High-End,Email,113.73239436619718
00110e08-8611-4fb4-9773-c27fc617bff6,2022-03-01,21,0.65625,0.34375,1.6521739130434783,17.0,23,0.0,2310.7999954223633,100.46956501836362,0.0,2310.7999954223633,100.46956501836362,0.0,lunch,3014.400001525879,94.20978289065154,0.0,7.0,4.0,12.0,0.0,23.0,0.0,0.3,0.52,0.17,0.0,32,21,1,0.0,High-End,Email,117.3
0011b87c-15f9-43f0-8f89-bfd38f2ee308,2022-03-01,23,0.6764705882352942,0.32352941176470584,2.1875,16.0,16,0.0,1608.4999923706055,100.53124952316284,0.0,1608.4999923706055,100.53124952316284,0.0,lunch,2520.099983215332,95.84562430381774,0.0,2.0,3.0,8.0,3.0,16.0,0.0,0.13,0.5,0.19,0.19,34,23,1,1.0,Low-End,Organic,120.02358490566037
0020e9da-26d2-4c6c-afd7-6b43a318eb2a,2022-02-01,3,0.375,0.625,8.5,19.0,4,0.0,279.4000015258789,69.85000038146973,0.0,279.4000015258789,69.85000038146973,0.0,dinner,279.4000015258789,69.85000038146973,0.0,2.0,2.0,0.0,0.0,4.0,0.0,0.5,0.0,0.5,0.0,8,3,1,1.0,Low-End,SMS,129.20454545454547
0027ab3d-c2e0-44a7-a6ac-560709f94630,2022-03-01,12,0.3870967741935484,0.6129032258064516,2.066666666666667,16.0,15,0.0,1483.2999954223633,98.88666636149088,0.0,1483.2999954223633,98.88666636149088,0.0,lunch,2421.7999877929688,96.36833279927572,0.0,1.0,3.0,8.0,3.0,15.0,0.0,0.07,0.53,0.2,0.2,31,12,1,0.0,High-End,Meta,116.11801242236025
002b30cc-ce89-4415-a874-6cb448e24b21,2022-03-01,0,0.0,1.0,null,21.0,1,0.0,77.4000015258789,77.4000015258789,0.0,77.4000015258789,77.4000015258789,0.0,dinner,77.4000015258789,77.4000015258789,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,9,0,0,0.0,Low-End,Meta,105.6923076923077
002eaf8c-095d-4516-bbd2-8456c863dafc,2022-03-01,2,0.1111111111111111,0.8888888888888888,7.333333333333333,17.0,3,0.0,200.99999618530273,66.99999872843425,0.0,200.99999618530273,66.99999872843425,0.0,lunch,337.1999931335449,67.54999860127768,0.0,2.0,1.0,0.0,0.0,3.0,0.0,0.67,0.0,0.33,0.0,18,2,1,0.0,Low-End,Meta,106.87654320987654
0033d257-02da-4237-8d57-6fd3f7f955df,2022-03-01,1,0.047619047619047616,0.9523809523809523,4.285714285714286,19.0,7,0.0,420.3000030517578,60.04285757882254,0.0,420.3000030517578,60.04285757882254,0.0,dinner,566.9000015258789,66.67142840794155,0.0,2.0,2.0,0.0,3.0,7.0,0.0,0.29,0.0,0.29,0.43,21,1,1,1.0,Low-End,Email,107.88775510204081


In [0]:
input_df.columns

Out[164]: ['customer_id',
 'month',
 'conversions',
 'cvr',
 'drop_rate',
 'avg_elapsed_time',
 'avg_hour',
 'frequency',
 'tot_discount_percentage',
 'tot_gross_value',
 'avg_expenses',
 'avg_discount',
 'tot_net_paid',
 'avg_paid_value',
 'avg_discount_percentage',
 'avg_shift',
 'last3_months_exp',
 'last3_months_avg_exp',
 'Pizza',
 'Burger',
 'Alcohol',
 'Japanese',
 'Veggy',
 'tot_values',
 '%Pizza',
 '%Burger',
 '%Japanese',
 '%Alcohol',
 '%Veggy',
 'freq_sessions',
 'bought',
 'response']

In [0]:
input_df\
    .select(f.col('response'), f.col('month'))\
    .groupBy(f.col('month'), f.col('response'))\
    .agg(f.count(f.col('response'))).display()

month,response,count(response)
2022-02-01,0,5501
2022-02-01,1,38178
2022-03-01,0,5714
2022-03-01,1,51294
2022-04-01,0,5569
2022-04-01,1,47915


### 2.2 - Correlation check

In [0]:

# create copy of input_data to create a correlation matrix. 

corr_df = input_df.alias('corr_df')

corr_df = (
    corr_df
    .withColumn('lunch_shift', f.when(f.col('avg_shift') == 'lunch', f.lit(1)).otherwise(f.lit(0)))
    .withColumn('dinner_shift', f.when(f.col('avg_shift') == 'dinner', f.lit(1)).otherwise(f.lit(0)))
    .withColumn('breakfast_shift', f.when(f.col('avg_shift') == 'breakfast', f.lit(1)).otherwise(f.lit(0)))
    .withColumn('low_end_type', f.when(f.col('device_type') == 'Low-End', f.lit(1)).otherwise(f.lit(0)))
    .withColumn('high_end_type', f.when(f.col('device_type') == 'High-End', f.lit(1)).otherwise(f.lit(0)))
    .withColumn('org_origin', f.when(f.col('install_origin') == 'Organic', f.lit(1)).otherwise(f.lit(0)))
    .withColumn('email_origin', f.when(f.col('install_origin') == 'Email', f.lit(1)).otherwise(f.lit(0)))
    .withColumn('sms_origin', f.when(f.col('install_origin') == 'SMS', f.lit(1)).otherwise(f.lit(0)))
    .withColumn('meta_origin', f.when(f.col('install_origin') == 'Meta', f.lit(1)).otherwise(f.lit(0)))
)

corr_df.printSchema()
    



root
 |-- customer_id: string (nullable = true)
 |-- month: date (nullable = true)
 |-- conversions: long (nullable = true)
 |-- cvr: double (nullable = true)
 |-- drop_rate: double (nullable = true)
 |-- avg_elapsed_time: double (nullable = true)
 |-- avg_hour: double (nullable = true)
 |-- frequency: long (nullable = true)
 |-- tot_discount_percentage: double (nullable = true)
 |-- tot_gross_value: double (nullable = true)
 |-- avg_expenses: double (nullable = true)
 |-- avg_discount: double (nullable = true)
 |-- tot_net_paid: double (nullable = true)
 |-- avg_paid_value: double (nullable = true)
 |-- avg_discount_percentage: double (nullable = true)
 |-- avg_shift: string (nullable = true)
 |-- last3_months_exp: double (nullable = true)
 |-- last3_months_avg_exp: double (nullable = true)
 |-- Pizza: float (nullable = true)
 |-- Burger: float (nullable = true)
 |-- Alcohol: float (nullable = true)
 |-- Japanese: float (nullable = true)
 |-- Veggy: float (nullable = true)
 |-- tot_va

In [0]:
list = ['avg_shift', 'device_type', 'install_origin', 'month', 'customer_id', 'high_end_type']
  
# delete two columns
corr_df = corr_df.drop(*list)

In [0]:
from pyspark.mllib.stat import Statistics
import pandas as pd

# result can be used w/ seaborn's heatmap
def compute_correlation_matrix(corr_df, method='pearson'):
    # wrapper around
    # https://forums.databricks.com/questions/3092/how-to-calculate-correlation-matrix-with-all-colum.html
    df_rdd = corr_df.rdd.map(lambda row: row[0:])
    corr_mat = Statistics.corr(df_rdd, method=method)
    corr_mat_df = pd.DataFrame(corr_mat,
                    columns=corr_df.columns, 
                    index=corr_df.columns)
    return corr_mat_df

In [0]:
from pyspark.sql import DataFrame

new_df = compute_correlation_matrix(corr_df, method='spearman')
new_df

# add markdowns on correlated (over 70/80, -70/80)

,conversions,cvr,drop_rate,avg_elapsed_time,avg_hour,frequency,tot_discount_percentage,tot_gross_value,avg_expenses,avg_discount,...,is_referee,duration_sec,lunch_shift,dinner_shift,breakfast_shift,low_end_type,org_origin,email_origin,sms_origin,meta_origin
conversions,1.000000,0.711041,-0.711041,-0.762653,-0.205896,0.789520,0.102541,0.755379,0.420950,0.090701,...,-0.139959,0.328404,0.267140,-0.265812,-0.022268,-0.237766,0.154155,-0.201295,0.108053,-0.076883
cvr,0.711041,1.000000,-1.000000,-0.518023,-0.222043,0.527747,0.121077,0.395497,-0.001923,0.116998,...,-0.198052,0.483473,0.233447,-0.232619,-0.014281,-0.407503,0.030152,-0.203271,0.184843,-0.040005
drop_rate,-0.711041,-1.000000,1.000000,0.518023,0.222043,-0.527747,-0.121077,-0.395497,0.001923,-0.116998,...,0.198052,-0.483473,-0.233447,0.232619,0.014281,0.407503,-0.030152,0.203271,-0.184843,0.040005
avg_elapsed_time,-0.762653,-0.518023,0.518023,1.000000,0.190608,-0.903888,-0.129100,-0.827404,-0.387261,-0.116408,...,0.123229,-0.284030,-0.263138,0.261540,0.026433,0.206589,-0.149625,0.186174,-0.093911,0.071495
avg_hour,-0.205896,-0.222043,0.222043,0.190608,1.000000,-0.195183,-0.036593,-0.182374,-0.084452,-0.034479,...,0.100701,-0.117869,-0.853799,0.857686,-0.055196,0.190761,-0.042965,0.095040,-0.099683,0.057264
frequency,0.789520,0.527747,-0.527747,-0.903888,-0.195183,1.000000,0.147679,0.923554,0.434683,0.134099,...,-0.128383,0.288456,0.276378,-0.274546,-0.030150,-0.215110,0.156263,-0.190835,0.094536,-0.074169
tot_discount_percentage,0.102541,0.121077,-0.121077,-0.129100,-0.036593,0.147679,1.000000,0.110914,0.006656,0.998306,...,-0.029514,0.072755,0.045658,-0.045413,-0.004094,-0.060174,0.007057,-0.025973,0.029214,-0.013412
tot_gross_value,0.755379,0.395497,-0.395497,-0.827404,-0.182374,0.923554,0.110914,1.000000,0.718001,0.097338,...,-0.096739,0.206413,0.263450,-0.261583,-0.030625,-0.131120,0.208050,-0.207401,0.065304,-0.078052
avg_expenses,0.420950,-0.001923,0.001923,-0.387261,-0.084452,0.434683,0.006656,0.718001,1.000000,-0.003885,...,0.007412,-0.024257,0.141515,-0.140435,-0.017649,0.089028,0.232505,-0.154980,-0.029940,-0.049978
avg_discount,0.090701,0.116998,-0.116998,-0.116408,-0.034479,0.134099,0.998306,0.097338,-0.003885,1.000000,...,-0.028515,0.071162,0.041967,-0.041748,-0.003660,-0.059252,0.003603,-0.023479,0.029679,-0.012847


### Correlation pairs 90% plus / -90% minus <br> <br>
- CVR vs Drop Rate (1.00) since it is the same information but from the opposite sides
- Conversions vs Bought
- Avg elapsed time (days since last order) vs Frequency (0.92)
- Avg elapsed time (days since last order) vs tot paid net (0.9)
- Frequency vs total gross / total net paid (0.90+)
- Total discount vs avg discount (0.99)
- avg expenses vs avg value paid / last 3 monhs avg expenses (0.99)
- Lunch shift vs dinner shift (0.99) - logically as almost noone except few customers have had breakfast orders the most

### Correlation pairs 80-90% / -80 -90% <br> <br>
- avg elapsed time (days since last order) vs total gross / total net paid (0.82)
- avg elapsed time vs last 3 months expenses (0.86)
- Avg hour vs shift (lunch / dinner) (0.86)
- last 3 month expenses vs frequency (0.85)
- last 3 month expenses vs total gross value (0.89)

### Correlation pairs 70-80% / -70 -80%<br> <br>
- conversions vs cvr / drop rate (0.71), vs frequency, vs tot gross value, vs tot net paid, vs tot values (0.79)
- cvr vs bought (0.71)
- avg elapsed time vs conversions, vs bought
- frequency vs bought
- total gross values vs avg expenses, avg paid value (0.71)

## Final Correlation Comments<br> <br>
As expected, many created variables are derived from others and therefore have a strong linear relationship. If keeping them, we would risk high multicolinearity, hurting the model, computational power and eventually even prediction results. Variables very close to each other, or redundant ones - such as drop out rate (w/ formula as 1 - conversion rate), should be extracted.

Instead of manually chosing all the highly correlated variables to extract from the final dataframe, in the next cells we go with the principal component analysis instead. This should ensure the best features' selection while keeping the most information needed for the modeling. We hope to go with around 10 to 14 PCA components (features).

# 3 - Pipeline

In [0]:
# split train test and prediction

train_ds      = input_df.filter(f.col('month').between('2022-02-01', '2022-03-31')) 
prediction_ds = input_df.filter(f.col('month') >= '2022-04-01');

train_data, test_data = train_ds.randomSplit([0.7, 0.3], 2022)

In [0]:
from pyspark.ml.feature import Imputer, VectorAssembler, StringIndexer, OneHotEncoder, MinMaxScaler, PCA
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline


numerical = ['conversions','cvr', 'drop_rate', 'avg_elapsed_time', 'avg_hour', 'frequency',
             'tot_discount_percentage', 'tot_gross_value', 'avg_expenses', 'avg_discount', 'tot_net_paid',
             'avg_paid_value', 'avg_discount_percentage', 'last3_months_exp','last3_months_avg_exp', 'Pizza', 'Burger',
             'Alcohol', 'Japanese', 'Veggy', 'tot_values', '%Pizza', '%Burger', '%Japanese',
             '%Alcohol', '%Veggy', 'freq_sessions', 'duration_sec','bought']
categorical = ['avg_shift', 'device_type', 'install_origin']

impute = Imputer(inputCols=['avg_elapsed_time', 'duration_sec'], outputCols=['avg_elapsed_time', 'duration_sec'])
assemble = VectorAssembler(inputCols = numerical, outputCol='continuous_features')
index = StringIndexer(inputCols=categorical, outputCols=['device_type_idx', 'avg_shift_idx', 'install_origin_idx'])
one_hot = OneHotEncoder(inputCols=['device_type_idx', 'avg_shift_idx', 'install_origin_idx'], 
                        outputCols=['device_type_vector','avg_shift_vector', 'install_origin_vector' ])
scale = MinMaxScaler(inputCol='continuous_features', outputCol='scaled_continuous_features')
final_assemble = VectorAssembler(inputCols=['scaled_continuous_features', 'device_type_vector',
                                            'avg_shift_vector', 'install_origin_vector', 'is_referee'], outputCol='features')
PCA   = PCA(k=7, inputCol= 'features', outputCol= 'pcaFeatures')
lr    = LogisticRegression(featuresCol="pcaFeatures", 
                           labelCol="response", 
                           predictionCol="prediction")


pipe = Pipeline()
pipe.setStages(
    [
        impute,
        assemble,
        index,
        one_hot,
        scale,
        final_assemble,
        PCA,
        lr
    ]
)

Out[28]: Pipeline_a14c3ba95442

# 4 - Modelling

In [0]:
pipe_model = pipe.fit(train_ds)

In [0]:
fitted_data = pipe_model.transform(train_ds)
fitted_data.display()

customer_id,month,conversions,cvr,drop_rate,avg_elapsed_time,avg_hour,frequency,tot_discount_percentage,tot_gross_value,avg_expenses,avg_discount,tot_net_paid,avg_paid_value,avg_discount_percentage,avg_shift,last3_months_exp,last3_months_avg_exp,Pizza,Burger,Alcohol,Japanese,Veggy,tot_values,%Pizza,%Burger,%Japanese,%Alcohol,%Veggy,freq_sessions,bought,response,is_referee,device_type,install_origin,duration_sec,continuous_features,device_type_idx,avg_shift_idx,install_origin_idx,device_type_vector,avg_shift_vector,install_origin_vector,scaled_continuous_features,features,pcaFeatures,rawPrediction,probability,prediction
0000006a-f50f-4153-aa42-e708216f7d66,2022-02-01,4,0.4,0.6,5.5,15.0,4,0.0,458.2000045776367,114.55000114440918,0.0,458.2000045776367,114.55000114440918,0.0,lunch,458.2000045776367,114.55000114440918,2.0,1.0,1.0,0.0,0.0,4.0,0.5,0.25,0.0,0.25,0.0,10,4,1,0.0,High-End,Meta,114.73584905660377,"Map(vectorType -> dense, length -> 29, values -> List(4.0, 0.4, 0.6, 5.5, 15.0, 4.0, 0.0, 458.2000045776367, 114.55000114440918, 0.0, 458.2000045776367, 114.55000114440918, 0.0, 458.2000045776367, 114.55000114440918, 2.0, 1.0, 1.0, 0.0, 0.0, 4.0, 0.5, 0.25, 0.0, 0.25, 0.0, 10.0, 114.73584905660377, 4.0))",0.0,0.0,0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 3, indices -> List(0), values -> List(1.0))","Map(vectorType -> dense, length -> 29, values -> List(0.08695652173913043, 0.4, 0.6, 0.016566265060240965, 0.6521739130434783, 0.06, 0.0, 0.010754015135181177, 0.08386924302525421, 0.0, 0.010754015135181177, 0.08386924302525421, 0.0, 0.008929627107530355, 0.08386924302525421, 0.05263157894736842, 0.043478260869565216, 0.05555555555555555, 0.0, 0.0, 0.06, 0.5, 0.25, 0.0, 0.25, 0.0, 0.14516129032258063, 0.37524262507213174, 0.08695652173913043))","Map(vectorType -> dense, length -> 36, values -> List(0.08695652173913043, 0.4, 0.6, 0.016566265060240965, 0.6521739130434783, 0.06, 0.0, 0.010754015135181177, 0.08386924302525421, 0.0, 0.010754015135181177, 0.08386924302525421, 0.0, 0.008929627107530355, 0.08386924302525421, 0.05263157894736842, 0.043478260869565216, 0.05555555555555555, 0.0, 0.0, 0.06, 0.5, 0.25, 0.0, 0.25, 0.0, 0.14516129032258063, 0.37524262507213174, 0.08695652173913043, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0))","Map(vectorType -> dense, length -> 7, values -> List(-0.7192454605614111, 0.11117110523724041, -0.8868512246013738, 0.19568962927055236, -0.7114062783561349, -0.08025613885802846, 0.5263915901282261))","Map(vectorType -> dense, length -> 2, values -> List(-3.278214817333115, 3.278214817333115))","Map(vectorType -> dense, length -> 2, values -> List(0.036326157791163285, 0.9636738422088367))",1.0
0010cba0-318f-43e9-94e7-c0638c33c80b,2022-03-01,35,0.6862745098039216,0.3137254901960784,1.24,15.0,25,73.6500015258789,6045.0,241.8,2.946000061035156,5971.349998474121,238.85399993896485,0.012183623081204121,lunch,34556.24977874756,769.083813157515,11.0,3.0,8.0,0.0,3.0,25.0,0.44,0.12,0.0,0.32,0.12,51,35,1,0.0,High-End,Meta,117.18275862068965,"Map(vectorType -> dense, length -> 29, values -> List(35.0, 0.6862745098039216, 0.3137254901960784, 1.24, 15.0, 25.0, 73.6500015258789, 6045.0, 241.8, 2.946000061035156, 5971.349998474121, 238.85399993896485, 0.012183623081204121, 34556.24977874756, 769.083813157515, 11.0, 3.0, 8.0, 0.0, 3.0, 25.0, 0.44, 0.12, 0.0, 0.32, 0.12, 51.0, 117.18275862068965, 35.0))",0.0,0.0,0.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 3, indices -> List(0), values -> List(1.0))","Map(vectorType -> dense, length -> 29, values -> List(0.7608695652173912, 0.6862745098039216, 0.3137254901960784, 0.0037349397590361448, 0.6521739130434783, 0.48, 0.11217729479130402, 0.1433235894978177, 0.18128924272

# 5 - Model Evaluation and optimization

In [0]:

from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="response",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC",
)

metric = evaluator.evaluate(fitted_data)
print(f"Area under ROC = {metric} ")

Area under ROC = 0.8760004913354661 


In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = (
    ParamGridBuilder()
    .addGrid(lr.regParam, [0.5, 0.2, 0.1, 0.01])
    .addGrid(lr.elasticNetParam, [0.0,0.2, 0.25 , 0.5,0.8, 1.0]) 
    .build()
)

In [0]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: pcaFeatures)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: response)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimizatio

In [0]:
!pip install mlflow --quiet

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
# If you get the error "YOU HAVENT CONFIGURED YOUR CLI", run this:
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
dbutils.fs.put("file:///root/.databrickscfg","[DEFAULT]\nhost=https://community.cloud.databricks.com\ntoken = "+token,overwrite=True)

Wrote 98 bytes.
Out[35]: True

In [0]:
from pyspark.ml.tuning import CrossValidator
import mlflow
from mlflow import spark

mlflow.pyspark.ml.autolog()
 
mlflow.start_run()
cv = CrossValidator(
    estimator=pipe,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=2
)
 
cv_model = cv.fit(train_ds)

2022/06/21 21:11:17 WARNING mlflow.utils: Truncated the value of the key `VectorAssembler_1.inputCols`. Truncated value: `['conversions', 'cvr', 'drop_rate', 'avg_elapsed_time', 'avg_hour', 'frequency', 'tot_discount_percentage', 'tot_gross_value', 'avg_expenses', 'avg_discount', 'tot_net_paid', 'avg_paid_value', 'avg_discount_percentage', 'last3_months_exp', 'last3_...`
2022/06/21 21:11:18 WARNING mlflow.utils: Truncated the value of the key `VectorAssembler_1.inputCols`. Truncated value: `['conversions', 'cvr', 'drop_rate', 'avg_elapsed_time', 'avg_hour', 'frequency', 'tot_discount_percentage', 'tot_gross_value', 'avg_expenses', 'avg_discount', 'tot_net_paid', 'avg_paid_value', 'avg_discount_percentage', 'last3_months_exp', 'last3_...`
2022/06/21 21:11:19 WARNING mlflow.utils: Truncated the value of the key `VectorAssembler_1.inputCols`. Truncated value: `['conversions', 'cvr', 'drop_rate', 'avg_elapsed_time', 'avg_hour', 'frequency', 'tot_discount_percentage', 'tot_gross_value', 'av

In [0]:
mlflow.spark.log_model(cv_model.bestModel, "model-file")# logs model as artifacts
mlflow.end_run()

In [0]:
print(cv_model.avgMetrics)

[0.8743949119663307, 0.5, 0.5, 0.5, 0.5, 0.5, 0.8748873233763264, 0.8342385404436317, 0.813335367113174, 0.5, 0.5, 0.5, 0.8757202788306094, 0.8606856021326458, 0.8562261458647044, 0.8132700513264455, 0.7524681727164235, 0.5, 0.8802088323193349, 0.8796882334541194, 0.8795441171846625, 0.8786058298598327, 0.8770023415088206, 0.8755978395362136]


In [0]:
best_model = cv_model.bestModel

In [0]:
fitted_test_data = best_model.transform(test_data)

In [0]:
train_metric = evaluator.evaluate(fitted_data)
test_metric = evaluator.evaluate(fitted_test_data)

print(f"Area under ROC on TRAIN= {train_metric}")
print(f"Area under ROC on TEST= {test_metric}")

Area under ROC on TRAIN= 0.8760015462188199
Area under ROC on TEST= 0.875340577791595


# 6 - Evaluation of model on April dataset and result extraction

In [0]:
# get the predictions for the test dataset with customers in april

fitted_prediction_data = best_model.transform(prediction_ds)
predict_metric = evaluator.evaluate(fitted_prediction_data)


In [0]:
print(f"Area under ROC on TRAIN= {predict_metric}")

Area under ROC on TRAIN= 0.8682853627998809


In [0]:
fitted_prediction_data.limit(100).display()

customer_id,month,conversions,cvr,drop_rate,avg_elapsed_time,avg_hour,frequency,tot_discount_percentage,tot_gross_value,avg_expenses,avg_discount,tot_net_paid,avg_paid_value,avg_discount_percentage,avg_shift,last3_months_exp,last3_months_avg_exp,Pizza,Burger,Alcohol,Japanese,Veggy,tot_values,%Pizza,%Burger,%Japanese,%Alcohol,%Veggy,freq_sessions,bought,response,is_referee,device_type,install_origin,duration_sec,continuous_features,device_type_idx,avg_shift_idx,install_origin_idx,device_type_vector,avg_shift_vector,install_origin_vector,scaled_continuous_features,features,pcaFeatures,rawPrediction,probability,prediction
0007ae88-2bc4-4a19-a8d5-88f7c438508f,2022-04-01,1,0.05,0.95,16.0,16.0,2,0.0,106.69999694824219,53.349998474121094,0.0,106.69999694824219,53.349998474121094,0.0,lunch,292.60000228881836,57.658333460489914,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,20,1,1,0.0,Low-End,Organic,114.49438202247191,"Map(vectorType -> sparse, length -> 29, indices -> List(0, 1, 2, 3, 4, 5, 7, 8, 10, 11, 13, 14, 17, 20, 24, 26, 27, 28), values -> List(1.0, 0.05, 0.95, 16.0, 16.0, 2.0, 106.69999694824219, 53.349998474121094, 106.69999694824219, 53.349998474121094, 292.60000228881836, 57.658333460489914, 2.0, 2.0, 1.0, 20.0, 114.49438202247191, 1.0))",0.0,1.0,2.0,"Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())","Map(vectorType -> sparse, length -> 3, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 29, indices -> List(0, 1, 2, 3, 4, 5, 7, 8, 10, 11, 13, 14, 17, 20, 24, 26, 27, 28), values -> List(0.021739130434782608, 0.05, 0.95, 0.04819277108433735, 0.6956521739130435, 0.02, 0.0024132464593607907, 0.037015771154135506, 0.0024132464593607907, 0.037015771154135506, 0.005666727163777154, 0.04031414441046853, 0.1111111111111111, 0.02, 1.0, 0.3064516129032258, 0.37345232268746553, 0.021739130434782608))","Map(vectorType -> sparse, length -> 36, indices -> List(0, 1, 2, 3, 4, 5, 7, 8, 10, 11, 13, 14, 17, 20, 24, 26, 27, 28, 29, 34), values -> List(0.021739130434782608, 0.05, 0.95, 0.04819277108433735, 0.6956521739130435, 0.02, 0.0024132464593607907, 0.037015771154135506, 0.0024132464593607907, 0.037015771154135506, 0.005666727163777154, 0.04031414441046853, 0.1111111111111111, 0.02, 1.0, 0.3064516129032258, 0.37345232268746553, 0.021739130434782608, 1.0, 1.0))","Map(vectorType -> dense, length -> 7, values -> List(-0.16386745758028462, 0.924459730174394, 0.3598691279215309, -0.52877128048704, 0.12327021612587054, 0.2448045347253444, 0.45323004452270577))","Map(vectorType -> dense, length -> 2, values -> List(-0.888473714147203, 0.888473714147203))","Map(vectorType -> dense, length -> 2, values -> List(0.2914248996469466, 0.7085751003530534))",1.0
001b3652-478d-4e35-aca8-69dfbe4e877e,2022-04-01,3,0.5,0.5,7.333333333333333,18.0,3,0.0,105.20000076293945,35.06666692097982,0.0,105.20000076293945,35.06666692097982,0.0,dinner,352.9000015258789,39.21111128065322,0.0,3.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,6,3,1,1.0,High-End,SMS,102.06896551724138,"Map(vectorType -> sparse, length -> 29, indices -> List(0, 1, 2, 3, 4, 5, 7, 8, 10, 11, 13, 14, 16, 20, 22, 26, 27, 28), values -> List(3.0, 0.5, 0.5, 7.333333333333333, 18.0, 3.0, 105.20000076293945, 35.06666692097982, 105.20000076293945, 35.06666692097982, 352.9000015258789, 39.21111128065322, 3.0, 3.0, 1.0, 6.0, 102.06896551724138, 3.0))",1.0,0.0,1.0,"Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 3, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 29, indices -> List(0, 1, 2, 3, 4, 5, 7, 8, 10, 11, 13, 14, 16, 20, 22, 26, 27, 28), values -> List(0.06521739130434782, 0.5, 0.5, 0.02208835341365462, 0.7826086956521738, 0.04, 0.002377652943216648, 0.02301842600285286, 0.002377652943216648, 

In [0]:
fitted_prediction_data\
    .groupBy(f.col('prediction'))\
    .agg(
        f.sum(f.col('prediction')).alias('converted'),
        f.count(f.col('prediction')).alias('total')
        ).display()

prediction,converted,total
0.0,0.0,1437
1.0,52047.0,52047


In [0]:
fitted_prediction_data\
    .groupBy(f.col('response'))\
    .agg(
        f.sum(f.col('response')).alias('converted'),
        f.count(f.col('response')).alias('total')
        ).display()

response,converted,total
1,47915,47915
0,0,5569


In [0]:
from pyspark.ml.functions import vector_to_array

results_df = (
    fitted_prediction_data\
        .select(f.col('customer_id'), f.col('probability'), f.col('prediction'))\
        .withColumn("xs", vector_to_array("probability"))\
        .withColumn('probability_0' , f.col('xs')[0])\
        .withColumn('probability_1' , f.col('xs')[1])\
        .select(f.col('customer_id'), f.col('probability_1'), f.col('prediction').alias('is_target'))
)


In [0]:
# export file
results_df.display()


customer_id,probability_1,is_target
0007ae88-2bc4-4a19-a8d5-88f7c438508f,0.7085751003530534,1.0
001b3652-478d-4e35-aca8-69dfbe4e877e,0.9489556988676205,1.0
001eb828-0a75-40e6-84a4-ee6de0f5435a,0.6103599802021578,1.0
002018a8-f8ca-4749-9bc3-fe6697658a3b,0.9863502338983619,1.0
00308fd9-5c8d-472e-a8de-893aa0a90d93,0.9466722749768457,1.0
00514270-1d23-401a-9c3d-c65e97d614bd,0.988060151374458,1.0
00601f9f-03e4-41aa-92db-662b84ea95ac,0.7746988230932139,1.0
0063a7d3-aade-4dca-b4e0-bbe0439bebaf,0.968589664255155,1.0
006431cc-ced0-42fc-acf6-8617c5aecd6d,0.7716309453629518,1.0
0065bd3b-9138-48e6-8b90-c94d0c6d2242,0.9914199357274404,1.0


# 7 - Model performance evaluation

By comparing the outcome of the A/B test we see that in 47.8K cases we have conversions and on 5.6K we don't have. <br>
While with the model application, we can see that actually 52K customer could convert with probability over 50% and only 1.4K would not convert. <br>

In [0]:
print(f'We could say then the the overall uplift from the model application is {52047 / 47915}')

We could say then the the overall uplift from the model application is 1.0862360429927997


# 8 - Final comment

Overall the model improved the outcome of the conversions given the data that we have. COmpared to the random split of baseline, it performs better. 
We still think that further improvement can be done using other features, which would make the model more accurate. 
Another check could be done on F1 score and Recall, for a second iteration, as well as trying with other models, like Random Forest and Neural Network.